In [2]:
# Imports Section :
import GEOparse
import pandas as pd

In [3]:
# Loading Gene Expression data set : 
gse = GEOparse.get_GEO(filepath="../DataSets/GSE63060_family.soft.gz")

17-Nov-2022 12:42:46 INFO GEOparse - Parsing ../DataSets/GSE63060_family.soft.gz: 
17-Nov-2022 12:42:46 DEBUG GEOparse - DATABASE: GeoMiame
17-Nov-2022 12:42:46 DEBUG GEOparse - SERIES: GSE63060
17-Nov-2022 12:42:46 DEBUG GEOparse - PLATFORM: GPL6947
17-Nov-2022 12:42:46 DEBUG GEOparse - SAMPLE: GSM1539080
17-Nov-2022 12:42:46 DEBUG GEOparse - SAMPLE: GSM1539081
17-Nov-2022 12:42:46 DEBUG GEOparse - SAMPLE: GSM1539082
17-Nov-2022 12:42:46 DEBUG GEOparse - SAMPLE: GSM1539083
17-Nov-2022 12:42:46 DEBUG GEOparse - SAMPLE: GSM1539084
17-Nov-2022 12:42:46 DEBUG GEOparse - SAMPLE: GSM1539085
17-Nov-2022 12:42:47 DEBUG GEOparse - SAMPLE: GSM1539086
17-Nov-2022 12:42:47 DEBUG GEOparse - SAMPLE: GSM1539087
17-Nov-2022 12:42:47 DEBUG GEOparse - SAMPLE: GSM1539088
17-Nov-2022 12:42:47 DEBUG GEOparse - SAMPLE: GSM1539089
17-Nov-2022 12:42:47 DEBUG GEOparse - SAMPLE: GSM1539090
17-Nov-2022 12:42:47 DEBUG GEOparse - SAMPLE: GSM1539091
17-Nov-2022 12:42:47 DEBUG GEOparse - SAMPLE: GSM1539092
17-Nov-2

In [ ]:
# Pre-Processing of data set :
gpl_lookup = []

for gpl_name, gpl in gse.gpls.items():
    gpl_lookup = gpl.table.filter(["ID", "ILMN_Gene"]).set_index("ID")["ILMN_Gene"].to_dict()

column_names = ["sample_id", "status", "ethinicity", "age", "gender", "included_in_case_study", "tissue"]
values = []
for gsm_name, gsm in gse.gsms.items():
    row = {}
    row[column_names[0]] = gsm_name

    for characteristic in gsm.metadata["characteristics_ch1"]:
        chsplit = characteristic.split(": ")
        row[chsplit[0]] = chsplit[1]
    
    tempdf = gsm.table.copy(deep=True)
    tempdf["ID_REF"] = tempdf["ID_REF"].apply(lambda id_ref : gpl_lookup[id_ref])
    tempdf = tempdf["VALUE"].groupby(tempdf["ID_REF"])
    for key, group in tempdf:
        row[key] = group.median()

    values.append(row)

print(values[1:5])




In [6]:
dataset = pd.DataFrame.from_dict(values)

In [9]:
dataset.head()

,sample_id,status,ethnicity,age,gender,included in case -control study,tissue,A1BG,A1CF,A26A1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,GSM1539080,MCI,Western European,65,Female,yes,blood,7.600951,7.439318,7.467857,...,7.439235,7.436031,7.386264,7.464524,8.046962,7.462498,10.618034,11.736463,8.561519,7.525411
1,GSM1539081,MCI,Other Caucasian,66,Female,yes,blood,7.672496,7.469146,7.529688,...,7.391362,7.616271,7.383703,7.756510,8.021271,7.472825,11.289126,11.930612,8.697649,7.666053
2,GSM1539082,MCI,Western European,67,Female,yes,blood,7.538046,7.478494,7.476638,...,7.426011,7.435217,7.452354,7.560671,8.010362,7.435084,11.068854,11.626505,8.889266,7.771253
3,GSM1539083,MCI,Unknown,67,Female,yes,blood,7.577860,7.486963,7.528242,...,7.519947,7.464119,7.451809,7.528391,8.034714,7.377990,10.527572,11.954639,8.665815,7.696854
4,GSM1539084,MCI,Western European,67,Female,yes,blood,7.678150,7.446429,7.511275,...,7.489877,7.520285,7.381160,7.624256,7.990016,7.453386,10.833655,11.895018,8.785981,7.647086


In [8]:
dataset.to_csv("../DataSets/GSE63060.csv")